# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [315]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [316]:
# Save path to SFH_data set in a variable
city_data_file = "../WeatherPY/output_data/cities.csv"
city_data_df = pd.read_csv(city_data_file)
city_data_df.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,566.0000,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,5.660000e+02
mean,282.5000,21.394806,19.942297,60.933127,68.418728,51.975265,8.120230,1.587770e+09
std,163.5344,32.889363,90.044692,19.712075,22.196379,36.878553,5.784607,1.408598e+02
min,0.0000,-54.800000,-175.200000,-2.200000,6.000000,0.000000,0.340000,1.587769e+09
25%,141.2500,-4.040000,-60.557500,46.400000,56.000000,14.250000,4.000000,1.587770e+09
50%,282.5000,24.110000,24.550000,63.000000,73.000000,55.000000,7.090000,1.587770e+09
75%,423.7500,49.522500,102.852500,77.652500,85.000000,90.000000,11.247500,1.587770e+09
max,565.0000,77.480000,179.320000,104.000000,100.000000,100.000000,55.250000,1.587770e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [317]:
# generate some (latitude, longitude) pairs
locations = city_data_df[["Lat", "Lng"]]
center_map = ["0", "0"]

In [318]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure(map_type='ROADMAP')

heatmap_layer = gmaps.heatmap_layer(
    city_data_df[["Lat", "Lng"]], weights=city_data_df["Humidity"],
    max_intensity=100, point_radius=7.5)
fig = gmaps.figure(center=center_map, zoom_level=1.9, layout = {'width' : '100%', 'height' : '675px'})
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='675px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [327]:
#Narrow down the DataFrame to find your ideal weather condition. 
# A max temperature lower than 75 degrees but higher than 55.
#Humidity < 50%
#Wind speed less than 7.5 mph.
prime_weather_df = city_data_df[city_data_df["Max Temp"] <75]
prime_weather_df = prime_weather_df[prime_weather_df["Max Temp"]>55]
prime_weather_df = prime_weather_df[prime_weather_df["Humidity"]<40]
prime_weather_df = prime_weather_df[prime_weather_df["Wind Speed"]<7]
prime_weather_df.dropna()
prime_weather

ValueError: invalid literal for int() with base 10: '33.73'

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [328]:
#Store into variable named hotel_df
print(hotel_df.dtypes)

Unnamed: 0      int64
City           object
Lat            object
Lng            object
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
Location       object
Hotel Name     object
dtype: object


In [329]:
#Store into variable named hotel_df
hotel_df = prime_weather

for i, city in hotel_df.iterrows():
    # get lat, lng from df
    lat = hotel_df["Lat"]
    lng = hotel_df["Lng"]

    # PARAMS Dictionary updates each iteration
    params = {"location":(f'{lat}, {lng}'),
          "radius": 5000,
          "types": "lodging",
          "key": g_key}
    
    # Create url and make API request #Get the JSON from the response
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Location"] = response["results"][0]["plus_code"]["compound_code"]
        hotel_df.loc[index, "Hotel Lng"] = response["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "Hotel Lat"] = response["results"][0]["geometry"]["location"]["lat"]
    except:
        #print(f"Hotel not found in the {row['City']}, {row['Country']}...Skipping.")
        print("-------------------------------------------------------")

-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------
-------------------------------------------------------


In [330]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [331]:
# Add marker layer ontop of heat map
target_corrdinates
markers = gmaps.marker_layer(target_corrdinates)
fig.add_layer(markers)
# Display figure
fig

TypeError: unsupported operand type(s) for -: 'str' and 'str'